Import necessary pakages

In [1]:
using JuMP
using HiGHS

In [2]:
Tasks = ["A1", "A2", "A3", "A4"]
T = length(Tasks)
Servers = ["B1", "B2", "B3", "B4"]
S = length(Servers)
BigValue = 10000000000
c = [# Cost of processing the task i on server j. BigValue is task cannot assing
    6 10 2 11;
    8 5 BigValue 2;
    9 2 11 1;
    7 BigValue 3 7;
]

4×4 Matrix{Int64}:
 6           10            2  11
 8            5  10000000000   2
 9            2           11   1
 7  10000000000            3   7

Linear model

In [3]:
lp_model = Model(HiGHS.Optimizer)
@variable(lp_model, x[t=1:T, s=1:S]>=0)
@objective(lp_model, Min, sum(c[t,s] * x[t,s] for t=1:T for s=1:S))
@constraint(lp_model, [s=1:S], sum(x[t,s] for t=1:T) == 1)
@constraint(lp_model, [t=1:T], sum(x[t,s] for s=1:S) == 1)
print(lp_model)
optimize!(lp_model)

println("Termination status: $(termination_status(lp_model))")
if termination_status(lp_model) == MOI.OPTIMAL
    println("Optimal objective value: $(objective_value(lp_model))")
    for t=1:T
        for s=1:S
            if value(x[t,s])>0
                println("Task $(t) is assigned to server $(s)")
            end
        end
    end
else
    println("No optimal solution available")
end

Running HiGHS 1.9.0 (git hash: 66f735e60): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+00, 1e+10]
  Bound  [0e+00, 0e+00]
  RHS    [1e+00, 1e+00]
Presolving model
8 rows, 16 cols, 32 nonzeros  0s
7 rows, 16 cols, 28 nonzeros  0s
Presolve : Reductions: rows 7(-1); columns 16(-0); elements 28(-4)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 7(7) 0s
          6     1.3000000000e+01 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 6
Objective value     :  1.3000000000e+01
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00
Termination status: OPTIMAL
Optimal objective value: 13.0
Task 1 is assigned to server 3
Task 2 is assigned to server 4
Task 3 is assigned to server 2
Task 4 is assigned to server 1


Mixed Integer Model

In [4]:
ilp_model = Model(HiGHS.Optimizer)
@variable(ilp_model, x[t=1:T, s=1:S]>=0, Bin)
@objective(ilp_model, Min, sum(c[t,s] * x[t,s] for t=1:T for s=1:S))
@constraint(ilp_model, [s=1:S], sum(x[t,s] for t=1:T) == 1)
@constraint(ilp_model, [t=1:T], sum(x[t,s] for s=1:S) == 1)
print(ilp_model)
optimize!(ilp_model)

println("Termination status: $(termination_status(ilp_model))")
if termination_status(ilp_model) == MOI.OPTIMAL
    println("Optimal objective value: $(objective_value(ilp_model))")
    for t=1:T
        for s=1:S
            if value(x[t,s])>0
                println("Task $(t) is assigned to server $(s)")
            end
        end
    end
else
    println("No optimal solution available")
end

Running HiGHS 1.9.0 (git hash: 66f735e60): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+00, 1e+10]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+00]
Presolving model
8 rows, 16 cols, 32 nonzeros  0s
8 rows, 16 cols, 32 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   8 rows
   16 cols (16 binary, 0 integer, 0 implied int., 0 continuous)
   32 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0        

In [7]:
println("Ratio between optimal ILP and LP is: $(objective_value(ilp_model)/objective_value(lp_model))")

Ratio between optimal ILP and LP is: 1.0
